In [38]:
import pandas as pd
import numpy as np
import json
import geopandas
import geopy
import requests
from IPython.display import IFrame
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
from sklearn.cluster import KMeans
import shapely.geometry
import pyproj
import csv
import numbers
import math

In [39]:
LD_grid = pd.read_csv("grid.csv",engine='python',sep=',')
#print(LD_grid)
LD_grid_landuse = pd.read_csv("landandbuildings.csv",encoding = 'unicode_escape',engine='python',sep=',')

Clustering =  4 # Number of clustering

In [40]:
LD_grid_landuse

,OBJECTID,osm_id,code,fclass,name,longitude,latitude,grid
0,47,13860363,7202,park,Parc Andr¨¦ Malraux,2.225952,48.891701,"0,6"
1,48,13860363,7202,park,Parc Andr¨¦ Malraux,2.226190,48.890964,"0,6"
2,49,13860363,7202,park,Parc Andr¨¦ Malraux,2.226495,48.890873,"0,6"
3,50,13860363,7202,park,Parc Andr¨¦ Malraux,2.226418,48.890763,"0,5"
4,51,13860363,7202,park,Parc Andr¨¦ Malraux,2.226309,48.890612,"0,5"
...,...,...,...,...,...,...,...,...
17542,18734,965129420,1500,service,PX MOULIN 34 DP,2.230218,48.885498,"3,0"
17543,18735,965129420,1500,service,PX MOULIN 34 DP,2.230236,48.885508,"3,0"
17544,18736,965129420,1500,service,PX MOULIN 34 DP,2.230256,48.885495,"3,0"
17545,18737,965129420,1500,service,PX MOULIN 34 DP,2.230237,48.885484,"3,0"


In [41]:
print(LD_grid_landuse.groupby('grid').count())

      OBJECTID  osm_id  code  fclass  name  longitude  latitude
grid                                                           
0,0         10      10    10      10    10         10        10
0,10        48      48    48      48    48         48        48
0,11        18      18    18      18    18         18        18
0,12        30      30    30      30    30         30        30
0,2         13      13    13      13    13         13        13
...        ...     ...   ...     ...   ...        ...       ...
9,5         92      92    92      92    92         92        92
9,6         11      11    11      11    11         11        11
9,7         27      27    27      27    27         27        27
9,8         47      47    47      47    47         47        47
9,9         15      15    15      15    15         15        15

[246 rows x 7 columns]


In [42]:
# one hot encoding
LD_grid_landuse_onehot = pd.get_dummies(LD_grid_landuse[['fclass']], prefix="", prefix_sep="")

# add District column back to dataframe
LD_grid_landuse_onehot['grid'] = LD_grid_landuse['grid'] 

# move District column to the first column
fixed_columns = [LD_grid_landuse_onehot.columns[-1]] + list(LD_grid_landuse_onehot.columns[:-1])
LD_grid_landuse_onehot = LD_grid_landuse_onehot[fixed_columns]

LD_grid_landuse_onehot.head()

,grid,apartments,commercial,grass,house,office,public,school,allotments,apartments,...,retail,roof,school,scrub,semidetached_house,service,stadium,terrace,tower,university
0,"0,6",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"0,6",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"0,6",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"0,5",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"0,5",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
LD_grid_landuse_grouped = LD_grid_landuse_onehot.groupby('grid').mean().reset_index()
LD_grid_landuse_grouped

,grid,apartments,commercial,grass,house,office,public,school,allotments,apartments,...,retail,roof,school,scrub,semidetached_house,service,stadium,terrace,tower,university
0,"0,0",0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.800000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
1,"0,10",0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.604167,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2,"0,11",0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.388889,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3,"0,12",0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.066667,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
4,"0,2",0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,"9,5",0.0,0.206522,0.0,0.0,0.130435,0.0,0.0,0.0,0.000000,...,0.0,0.021739,0.0,0.000000,0.0,0.0,0.0,0.0,0.032609,0.0
242,"9,6",0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
243,"9,7",0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
244,"9,8",0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [44]:
# Find out the top 10 restaurant type in the district

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 6

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['grid']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most poi Type'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most poi Type'.format(ind+1))

# create a new dataframe
LD_grid_landuse_grouped_sorted = pd.DataFrame(columns=columns)
LD_grid_landuse_grouped_sorted['grid'] = LD_grid_landuse_grouped['grid']

for ind in np.arange(LD_grid_landuse_grouped.shape[0]):
    LD_grid_landuse_grouped_sorted.iloc[ind, 1:] = return_most_common_venues(LD_grid_landuse_grouped.iloc[ind, :], num_top_venues)

LD_grid_landuse_grouped_sorted

,grid,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type
0,"0,0",apartments,park,apartments,retail,house,industrial
1,"0,10",apartments,residential,grass,apartments,house,industrial
2,"0,11",residential,apartments,grass,office,apartments,house
3,"0,12",office,construction,apartments,apartments,residential,house
4,"0,2",residential,apartments,hotel,house,industrial,meadow
...,...,...,...,...,...,...,...
241,"9,5",office,commercial,commercial,office,tower,roof
242,"9,6",commercial,office,apartments,residential,house,industrial
243,"9,7",commercial,construction,apartments,grass,house,industrial
244,"9,8",commercial,construction,apartments,grass,house,industrial


In [45]:
# set number of clusters
kclusters = Clustering

# set the attributes used for clustering
LD_grid_landuse_grouped_clustering = LD_grid_landuse_grouped.drop(columns='grid')

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0)
kmeans.fit(LD_grid_landuse_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 3, 3, 3, 3, 3, 3])

In [46]:
# Merge the result into the district dataframe
# add clustering labels
LD_grid_landuse_grouped_sorted['Cluster Labels'] = kmeans.labels_
#HK_district_dish_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

LD_grid_merged = LD_grid

# merge df_grouped with df_data to add latitude/longitude for each neighbourhood
LD_grid_merged = LD_grid_merged.join(LD_grid_landuse_grouped_sorted.set_index('grid'), on='grid')
LD_grid_merged['Cluster Labels']=[int(val) if isinstance(val,numbers.Number) & (math.isnan(val)==False) else 0 for val in LD_grid_merged['Cluster Labels']]
LD_grid_merged_drop=LD_grid_merged.dropna()

In [47]:
LD_grid_merged_drop

,grid,longitude,latitude,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,Cluster Labels
0,"0,0",2.226390,48.885906,apartments,park,apartments,retail,house,industrial,0
1,"1,0",2.227802,48.885906,park,apartments,residential,hotel,house,industrial,3
2,"2,0",2.229214,48.885906,apartments,school,hotel,residential,house,industrial,0
3,"3,0",2.230626,48.885906,hotel,park,allotments,service,school,residential,3
4,"4,0",2.232038,48.885906,park,apartments,residential,hotel,house,industrial,3
...,...,...,...,...,...,...,...,...,...,...
255,"15,12",2.247568,48.897131,house,apartments,apartments,retail,industrial,meadow,3
256,"16,12",2.248980,48.897131,dormitory,apartments,residential,house,industrial,meadow,3
257,"17,12",2.250392,48.897131,apartments,house,dormitory,grass,apartments,residential,0
258,"18,12",2.251804,48.897131,house,apartments,garage,apartments,residential,industrial,0


In [54]:
LD_grid_merged_drop.to_csv('LD_grid_landuse_cluster.csv', index=False, encoding='utf-8')
LD_grid_merged_drop = pd.read_csv("LD_grid_landuse_cluster.csv",engine='python',sep=',')
print(LD_grid_merged_drop)

      grid  longitude   latitude 1st Most poi Type 2nd Most poi Type  \
0      0,0   2.226390  48.885906        apartments              park   
1      1,0   2.227802  48.885906              park        apartments   
2      2,0   2.229214  48.885906        apartments            school   
3      3,0   2.230626  48.885906             hotel              park   
4      4,0   2.232038  48.885906              park        apartments   
..     ...        ...        ...               ...               ...   
241  15,12   2.247568  48.897131             house        apartments   
242  16,12   2.248980  48.897131         dormitory        apartments   
243  17,12   2.250392  48.897131        apartments             house   
244  18,12   2.251804  48.897131             house        apartments   
245  19,12   2.253216  48.897131        apartments             house   

    3rd Most poi Type 4th Most poi Type 5th Most poi Type 6th Most poi Type  \
0          apartments            retail             hous

In [55]:
# create map
map_cluster = folium.Map(tiles='cartodbpositron', location=[48.89114492, 2.240388888], zoom_start=14.6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LD_grid_merged_drop['latitude'], LD_grid_merged_drop['longitude'], LD_grid_merged_drop['grid'], LD_grid_merged_drop['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_cluster)
       
map_cluster.save('ld_district_landuse_cluster.html')
display(IFrame('ld_district_landeuse_cluster.html', width = 1500, height = 1200))
print([cluster-1])

[-1]


In [50]:
LD_grid_merged_drop[LD_grid_merged_drop['Cluster Labels'] == 0]

,grid,longitude,latitude,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,Cluster Labels
0,"0,0",2.226390,48.885906,apartments,park,apartments,retail,house,industrial,0
2,"2,0",2.229214,48.885906,apartments,school,hotel,residential,house,industrial,0
6,"6,0",2.234861,48.885906,house,apartments,apartments,retail,industrial,meadow,0
8,"11,0",2.241921,48.885906,apartments,apartments,retail,house,industrial,meadow,0
9,"12,0",2.243333,48.885906,apartments,apartments,retail,house,industrial,meadow,0
10,"13,0",2.244745,48.885906,apartments,apartments,retail,house,industrial,meadow,0
11,"14,0",2.246157,48.885906,apartments,grass,apartments,residential,house,industrial,0
17,"2,1",2.229214,48.886841,apartments,apartments,retail,house,industrial,meadow,0
19,"4,1",2.232038,48.886841,apartments,office,apartments,retail,house,industrial,0
20,"5,1",2.233449,48.886841,apartments,park,apartments,retail,house,industrial,0


In [51]:
LD_grid_merged_drop[LD_grid_merged_drop['Cluster Labels'] == 1]

,grid,longitude,latitude,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,Cluster Labels
7,"9,0",2.239097,48.885906,grass,residential,hotel,house,industrial,meadow,1
15,"18,0",2.251804,48.885906,grass,residential,hotel,house,industrial,meadow,1
22,"7,1",2.236273,48.886841,grass,residential,hotel,house,industrial,meadow,1
24,"10,1",2.240509,48.886841,grass,forest,apartments,residential,house,industrial,1
27,"13,1",2.244745,48.886841,grass,apartments,office,apartments,residential,house,1
...,...,...,...,...,...,...,...,...,...,...
190,"1,10",2.227802,48.895260,grass,stadium,residential,hotel,house,industrial,1
196,"9,10",2.239097,48.895260,hotel,grass,office,residential,house,industrial,1
197,"10,10",2.240509,48.895260,grass,office,meadow,residential,hotel,house,1
198,"11,10",2.241921,48.895260,grass,office,park,scrub,meadow,apartments,1


In [52]:
LD_grid_merged_drop[LD_grid_merged_drop['Cluster Labels'] == 2]

,grid,longitude,latitude,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,Cluster Labels
13,"16,0",2.248980,48.885906,office,apartments,residential,hotel,house,industrial,2
14,"17,0",2.250392,48.885906,office,grass,residential,hotel,house,industrial,2
18,"3,1",2.230626,48.886841,office,apartments,residential,hotel,house,industrial,2
31,"18,1",2.251804,48.886841,tower,office,roof,apartments,recreation_ground,hotel,2
35,"3,2",2.230626,48.887777,office,apartments,residential,hotel,house,industrial,2
36,"4,2",2.232038,48.887777,office,apartments,residential,hotel,house,industrial,2
37,"5,2",2.233449,48.887777,office,apartments,apartments,retail,house,industrial,2
46,"14,2",2.246157,48.887777,office,recreation_ground,tower,apartments,residential,hotel,2
47,"15,2",2.247568,48.887777,office,hotel,grass,park,recreation_ground,residential,2
57,"5,3",2.233449,48.888712,school,office,apartments,residential,hotel,house,2


In [53]:
LD_grid_merged_drop[LD_grid_merged_drop['Cluster Labels'] == 3]

,grid,longitude,latitude,1st Most poi Type,2nd Most poi Type,3rd Most poi Type,4th Most poi Type,5th Most poi Type,6th Most poi Type,Cluster Labels
1,"1,0",2.227802,48.885906,park,apartments,residential,hotel,house,industrial,3
3,"3,0",2.230626,48.885906,hotel,park,allotments,service,school,residential,3
4,"4,0",2.232038,48.885906,park,apartments,residential,hotel,house,industrial,3
5,"5,0",2.233449,48.885906,garages,house,park,residential,hotel,industrial,3
12,"15,0",2.247568,48.885906,school,roof,apartments,service,office,residential,3
...,...,...,...,...,...,...,...,...,...,...
236,"10,12",2.240509,48.897131,house,house,grass,school,retail,meadow,3
237,"11,12",2.241921,48.897131,house,house,roof,park,school,residential,3
238,"12,12",2.243333,48.897131,house,house,apartments,garages,roof,meadow,3
241,"15,12",2.247568,48.897131,house,apartments,apartments,retail,industrial,meadow,3
